In [1]:
!pip install selenium webdriver_manager

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [3]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode if you don't need a GUI

# Initialize the WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Open the webpage
url = "https://www.iplt20.com/matches/results"
driver.get(url)
print(driver.title)

IPL 2024 Match Results | Latest Cricket Scores and updates


In [4]:
# Give the page some time to load
time.sleep(5)  # Adjust sleep time based on page load speed

# Find all match elements
matches = driver.find_elements(By.CSS_SELECTOR, 'li.ng-scope')
# Locate the <ul> element with id 'team_archive'
team_archive = driver.find_element(By.ID, 'team_archive')

# Get all <li> elements within the <ul> element
#matches = team_archive.find_elements(By.TAG_NAME, 'li')

In [5]:
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time

def extract_team_info(team_element):    
    try:
        code = team_element.find_element(By.CSS_SELECTOR, 'div.vn-teamCode h3').text
    except NoSuchElementException:
        code = 'N/A'
    
    try:
        score_element = team_element.find_element(By.CSS_SELECTOR, 'div.vn-teamTitle p')
        score = score_element.text if score_element else 'N/A'
    except NoSuchElementException:
        score = 'N/A'
    
    try:
        overs_element = team_element.find_element(By.CSS_SELECTOR, 'div.vn-teamTitle span.ov-display')
        overs = overs_element.text if overs_element else 'N/A'
    except NoSuchElementException:
        overs = 'N/A'
    
    return code, score, overs

# Wait for the page to load and the elements to be present
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'team_archive'))
    )
except TimeoutException:
    print("Page did not load in time or elements not found.")
    driver.quit()
    exit()

# Locate the <ul> element with id 'team_archive'
team_archive = driver.find_element(By.ID, 'team_archive')
matches = team_archive.find_elements(By.CSS_SELECTOR, 'li.ng-scope')

# Collect data in a list of dictionaries
match_data = []

for match in matches:
    try:
        match_order_element = match.find_element(By.CSS_SELECTOR, 'span.vn-matchOrder')
        match_order = match_order_element.text if match_order_element else 'N/A'
    except NoSuchElementException:
        match_order = 'N/A'
    
    try:
        venue_element = match.find_element(By.CSS_SELECTOR, 'span.vn-venueDet p')
        venue = venue_element.text if venue_element else 'N/A'
    except NoSuchElementException:
        venue = 'N/A'
    
    try:
        date_time_element = match.find_element(By.CSS_SELECTOR, 'div.vn-matchDateTime')
        match_date_time = date_time_element.text if date_time_element else 'N/A'
    except NoSuchElementException:
        match_date_time = 'N/A'
    
    try:
        result_element = match.find_element(By.CSS_SELECTOR, 'div.vn-ticketTitle')
        result = result_element.text if result_element else 'N/A'
    except NoSuchElementException:
        result = 'N/A'
    
    try:
        team_1 = match.find_element(By.CSS_SELECTOR, 'div.vn-shedTeam')
        team_2 = match.find_element(By.CSS_SELECTOR, 'div.vn-shedTeam.vn-team-2')
        
        team_1_code, team_1_score, team_1_overs = extract_team_info(team_1)
        team_2_code, team_2_score, team_2_overs = extract_team_info(team_2)
    except NoSuchElementException as e:
        print(f"Error finding team details: {e}")
        team_1_code = team_1_score = team_1_overs = 'N/A'
        team_2_code = team_2_score = team_2_overs = 'N/A'

    try:
        comments_element = match.find_element(By.CSS_SELECTOR, 'div.vn-ticketTitle')
        comments = comments_element.text if comments_element else 'N/A'
    except NoSuchElementException:
        comments = 'N/A'
    
    match_data.append({
        'Match Order': match_order,
        'Venue': venue,
        'Date & Time': match_date_time,
        'Result': result,
        'Team 1 Code': team_1_code,
        'Team 1 Score': team_1_score,
        'Team 1 Overs': team_1_overs,
        'Team 2 Code': team_2_code,
        'Team 2 Score': team_2_score,
        'Team 2 Overs': team_2_overs,
        'Comments': comments
    })

In [6]:
# Create a DataFrame
df = pd.DataFrame(match_data)

In [7]:
df.head()

,Match Order,Venue,Date & Time,Result,Team 1 Code,Team 1 Score,Team 1 Overs,Team 2 Code,Team 2 Score,Team 2 Overs,Comments
0,FINAL,"MA Chidambaram Stadium, Chennai","MAY, SUN 26 , 7:30 pm IST",KOLKATA KNIGHT RIDERS WON BY 8 WICKETS (WINNERS),KKR,114/2,(10.3 OV),SRH,113,(18.3 OV ),KOLKATA KNIGHT RIDERS WON BY 8 WICKETS (WINNERS)
1,QUALIFIER 2,"MA Chidambaram Stadium, Chennai","MAY, FRI 24 , 7:30 pm IST",SUNRISERS HYDERABAD WON BY 36 RUNS (QUALIFIED),SRH,175/9,(20.0 OV),RR,139/7,(20.0 OV ),SUNRISERS HYDERABAD WON BY 36 RUNS (QUALIFIED)
2,ELIMINATOR,"Narendra Modi Stadium, Ahmedabad","MAY, WED 22 , 7:30 pm IST",RAJASTHAN ROYALS WON BY 4 WICKETS (QUALIFIED),RR,174/6,(19.0 OV),RCB,172/8,(20.0 OV ),RAJASTHAN ROYALS WON BY 4 WICKETS (QUALIFIED)
3,QUALIFIER 1,"Narendra Modi Stadium, Ahmedabad","MAY, TUE 21 , 7:30 pm IST",KOLKATA KNIGHT RIDERS WON BY 8 WICKETS (QUALIF...,KKR,164/2,(13.4 OV),SRH,159,(19.3 OV ),KOLKATA KNIGHT RIDERS WON BY 8 WICKETS (QUALIF...
4,MATCH 70,"Barsapara Cricket Stadium, Guwahati","MAY, SUN 19 , 7:30 pm IST",MATCH ABANDONED,RR,N/A,N/A,KKR,N/A,N/A,MATCH ABANDONED


In [8]:
# Save the DataFrame to a CSV file
output_file = 'ipl_results.csv'
df.to_csv(output_file, index=False)

print(f"Data saved to {output_file}")

Data saved to ipl_results.csv


In [9]:
# Close the browser
driver.quit()